<a href="https://colab.research.google.com/github/guddlaraji/files/blob/main/Project_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install sqlite3
!pip install pandas

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 769 kB of archives.
After this operation, 1,874 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.5 [769 kB]
Fetched 769 kB in 1s (993 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 125080 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.5_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.5) ...
Setting up sqlite3 (3.37.2-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import sqlite3
import pandas as pd
import os

In [ ]:
conn = sqlite3.connect("my_database.db")

In [ ]:
# Define the correct path for the database
db_path = "/content/drive/MyDrive/my_database.db"

In [ ]:
db_folder = "/content/drive/MyDrive/Sqldb folder"
os.makedirs(db_folder, exist_ok=True)  # Ensure the folder exists
db_path = os.path.join(db_folder, "my_database.db")

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("Database connected at:", db_path)


Database connected at: /content/drive/MyDrive/Sqldb folder/my_database.db


**customer table**

In [ ]:
cursor.execute('''
CREATE TABLE CUSTOMER1 ( customer_id INT ,
             	customer_name VARCHAR(20),
             	Address 	VARCHAR(20),
             	state_code  VARCHAR(3) ,
             	Telephone   VARCHAR(10)	);

''')
conn.commit()

In [ ]:
def insert_CUSTOMER1 (customer_id,customer_name,Address,state_code,Telephone):
    cursor.execute("INSERT INTO CUSTOMER1 (customer_id,customer_name,Address,state_code,Telephone) VALUES (?,?, ?, ?,?)", (customer_id,customer_name,Address,state_code,Telephone))
    conn.commit()

# Insertion
insert_CUSTOMER1(123001,"Oliver", "225-5,Emeryville", "CA" , "1897614500")
insert_CUSTOMER1(123002,"George", "194-6,New brighton","MN" , "1897617000")

In [ ]:
pip install tabulate

In [ ]:
from tabulate import tabulate


# Fetch data
cursor.execute("SELECT * FROM CUSTOMER1 ")
rows = cursor.fetchall()

# Get column names
column_names = [desc[0] for desc in cursor.description]

# Print in tabular format
print(tabulate(rows, headers=column_names, tablefmt="grid"))

1.Print customer Id, customer name and average account_balance maintained by each customer for all of his/her accounts in the bank.(8 Rows)

In [ ]:
cursor.execute("DROP TABLE IF EXISTS BANK_ACCOUNT_TRANSACTION;")

query = """
CREATE TABLE BANK_ACCOUNT_TRANSACTION (
    TransactionID INTEGER PRIMARY KEY,
    AccountNumber INTEGER,
    Amount DECIMAL(10,2),
    TransactionDate DATE
);
"""
cursor.execute(query)


In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Create CUSTOMER table
cursor.execute("""
CREATE TABLE IF NOT EXISTS CUSTOMER (
    CustomerID INTEGER PRIMARY KEY,
    CustomerName TEXT NOT NULL
);
""")

# Create ACCOUNT table
cursor.execute("""
CREATE TABLE IF NOT EXISTS ACCOUNT (
    AccountNumber INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(10,2),
    FOREIGN KEY (CustomerID) REFERENCES CUSTOMER(CustomerID)
);
""")

conn.commit()
conn.close()


In [ ]:
conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

query = """
SELECT
    c.CustomerID,
    c.CustomerName,
    AVG(a.Account_Balance) AS Average_Balance
FROM
    CUSTOMER c
JOIN
    ACCOUNT a ON c.CustomerID = a.CustomerID
GROUP BY
    c.CustomerID, c.CustomerName;
"""

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()


In [ ]:
import sqlite3

# 1. Connect (new session each time)
conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# 2. Create CUSTOMER and ACCOUNT tables
cursor.execute("DROP TABLE IF EXISTS CUSTOMER;")
cursor.execute("DROP TABLE IF EXISTS ACCOUNT;")

cursor.execute("""
CREATE TABLE CUSTOMER (
    CustomerID INTEGER PRIMARY KEY,
    CustomerName TEXT
);
""")

cursor.execute("""
CREATE TABLE ACCOUNT (
    AccountNumber TEXT PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(10,2),
    FOREIGN KEY(CustomerID) REFERENCES CUSTOMER(CustomerID)
);
""")

# 3. Insert 8 customers
cursor.executescript("""
INSERT INTO CUSTOMER VALUES (1, "Ramesh");
INSERT INTO CUSTOMER VALUES (2, "Suresh");
INSERT INTO CUSTOMER VALUES (3, "Geeta");
INSERT INTO CUSTOMER VALUES (4, "Anita");
INSERT INTO CUSTOMER VALUES (5, "Rajesh");
INSERT INTO CUSTOMER VALUES (6, "Sunita");
INSERT INTO CUSTOMER VALUES (7, "Vikas");
INSERT INTO CUSTOMER VALUES (8, "Pooja");
""")

# 4. Insert accounts for each customer
cursor.executescript("""
INSERT INTO ACCOUNT VALUES ("4000-1956-3456", 1, 12000);
INSERT INTO ACCOUNT VALUES ("4000-1956-2001", 1, 18500);
INSERT INTO ACCOUNT VALUES ("4000-1956-3401", 2, 21000);
INSERT INTO ACCOUNT VALUES ("4000-1956-5102", 3, 19000);
INSERT INTO ACCOUNT VALUES ("4000-1956-5698", 4, 20000);
INSERT INTO ACCOUNT VALUES ("4000-1956-9977", 5, 17800);
INSERT INTO ACCOUNT VALUES ("5000-1700-6091", 6, 25000);
INSERT INTO ACCOUNT VALUES ("5000-1700-7791", 7, 19500);
INSERT INTO ACCOUNT VALUES ("9000-1700-7777-4321", 8, 16500);
""")

conn.commit()

# 5. Query → Customer ID, Name, Avg Balance
query = """
SELECT
    c.CustomerID,
    c.CustomerName,
    AVG(a.Account_Balance) AS Average_Balance
FROM CUSTOMER c
JOIN ACCOUNT a ON c.CustomerID = a.CustomerID
GROUP BY c.CustomerID, c.CustomerName;
"""
cursor.execute(query)
rows = cursor.fetchall()

# 6. Print formatted output
print(f"{'CustomerID':<12} {'CustomerName':<15} {'Average_Balance'}")
print("-" * 45)
for row in rows:
    print(f"{row[0]:<12} {row[1]:<15} {row[2]:.2f}")

conn.close()


2) Print customer_id , account_number and balance_amount , condition that if balance_amount is nil then assign transaction_amount  for account_type = "Credit Card"(4 Rows)

In [ ]:
query = """
CREATE TABLE IF NOT EXISTS BANK_ACCOUNT_TRANSACTION (
    Account_Number VARCHAR(19),
    Transaction_amount DECIMAL(18,2),
    Transcation_channel VARCHAR(18),
    Province VARCHAR(3),
    Transaction_Date DATE
);
"""
cursor.execute(query)
conn.commit()


In [ ]:
import sqlite3

# 1. Connect
conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# 2. Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS BANK_ACCOUNT_TRANSACTION;")
cursor.execute("DROP TABLE IF EXISTS ACCOUNT;")

# 3. Create BANK_ACCOUNT_TRANSACTION table
cursor.execute("""
CREATE TABLE BANK_ACCOUNT_TRANSACTION (
    Account_Number VARCHAR(19),
    Transaction_amount DECIMAL(18,2),
    Transcation_channel VARCHAR(18),
    Province VARCHAR(3),
    Transaction_Date DATE
);
""")

# 4. Insert transaction data
cursor.executescript("""
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-3456", -2000, "ATM withdrawl", "CA", "2020-01-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -4000, "POS-Walmart", "MN", "2020-02-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -1600, "UPI transfer", "MN", "2020-01-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -6000, "Bankers cheque", "CA", "2020-03-23");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -3000, "Net banking", "CA", "2020-04-24");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", 23000, "cheque deposit", "MN", "2020-03-15");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5000-1700-6091", 40000, "ECS transfer", "NY", "2020-02-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5000-1700-7791", 40000, "ECS transfer", "NY", "2020-02-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-3401", 8000, "Cash Deposit", "NY", "2020-01-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5102", -6500, "ATM withdrawal", "NY", "2020-03-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5698", -9000, "Cash Deposit", "NY", "2020-03-27");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-9977", 50000, "ECS transfer", "NY", "2020-01-16");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -5000, "POS-Walmart", "NY", "2020-02-17");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -8000, "Shopping Cart", "MN", "2020-03-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -2500, "Shopping Cart", "MN", "2020-04-21");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5800-1700-9800-7755", -9000, "POS-Walmart", "MN", "2020-04-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5890-1970-7706-8912", -11000, "Shopping Cart", "NY", "2020-03-12");
""")

# 5. Create ACCOUNT table with Credit Card accounts
cursor.execute("""
CREATE TABLE ACCOUNT (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(18,2),
    AccountType VARCHAR(20)
);
""")

# 6. Insert sample accounts (some Credit Card with NULL/0 balance)
cursor.executescript("""
INSERT INTO ACCOUNT VALUES ("4000-1956-3456", 1, 0, "Credit Card");
INSERT INTO ACCOUNT VALUES ("4000-1956-2001", 2, NULL, "Credit Card");
INSERT INTO ACCOUNT VALUES ("5000-1700-6091", 3, 15000, "Savings");
INSERT INTO ACCOUNT VALUES ("5000-1700-7791", 4, NULL, "Credit Card");
""")

conn.commit()

# 7. Query: balance_amount or transaction_amount if NULL/0 for Credit Card
query = """
SELECT
    a.CustomerID,
    a.AccountNumber,
    CASE
        WHEN a.Account_Balance IS NULL OR a.Account_Balance = 0
        THEN (SELECT Transaction_amount
              FROM BANK_ACCOUNT_TRANSACTION t
              WHERE t.Account_Number = a.AccountNumber
              LIMIT 1)
        ELSE a.Account_Balance
    END AS Balance_Amount
FROM ACCOUNT a
WHERE a.AccountType = 'Credit Card';
"""

cursor.execute(query)
rows = cursor.fetchall()

# 8. Print output
print(f"{'CustomerID':<12} {'AccountNumber':<20} {'Balance_Amount'}")
print("-" * 50)
for row in rows:
    print(f"{row[0]:<12} {row[1]:<20} {row[2]}")

conn.close()


3) Print account_number and balance_amount , transaction_amount,Transaction_Date from Bank_Account_Details and bank_account_transaction for all the transactions occurred during march,2020 and april, 2020(12 Rows)

In [ ]:
query = """
SELECT
    a.AccountNumber,
    a.Account_Balance AS Balance_Amount,
    t.Transaction_amount,
    t.Transaction_Date
FROM ACCOUNT a
JOIN BANK_ACCOUNT_TRANSACTION t
    ON a.AccountNumber = t.Account_Number
WHERE strftime('%Y-%m', t.Transaction_Date) IN ('2020-03', '2020-04');
"""


In [ ]:
cursor.execute("DROP TABLE IF EXISTS Bank_Account_Details;")

cursor.execute("""
CREATE TABLE Bank_Account_Details (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    Account_Balance DECIMAL(18,2)
);
""")

# Insert sample data (from your ACCOUNT table)
cursor.executescript("""
INSERT INTO Bank_Account_Details VALUES ("4000-1956-3456", 0);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-2001", NULL);
INSERT INTO Bank_Account_Details VALUES ("5000-1700-6091", 15000);
INSERT INTO Bank_Account_Details VALUES ("5000-1700-7791", NULL);
""")


In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


In [ ]:
import sqlite3

# 1. Connect to database
conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# 2. Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS ACCOUNT;")
cursor.execute("DROP TABLE IF EXISTS BANK_ACCOUNT_TRANSACTION;")

# 3. Create ACCOUNT table
cursor.execute("""
CREATE TABLE ACCOUNT (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    Account_Balance DECIMAL(18,2)
);
""")

# 4. Insert sample account balances
cursor.executescript("""
INSERT INTO ACCOUNT VALUES ("4000-1956-3456", 0);
INSERT INTO ACCOUNT VALUES ("4000-1956-2001", NULL);
INSERT INTO ACCOUNT VALUES ("5000-1700-6091", 15000);
INSERT INTO ACCOUNT VALUES ("5000-1700-7791", NULL);
INSERT INTO ACCOUNT VALUES ("4000-1956-5102", -6500);
INSERT INTO ACCOUNT VALUES ("4000-1956-5698", -9000);
INSERT INTO ACCOUNT VALUES ("4000-1956-9977", 50000);
INSERT INTO ACCOUNT VALUES ("9000-1700-7777-4321", NULL);
INSERT INTO ACCOUNT VALUES ("5800-1700-9800-7755", NULL);
INSERT INTO ACCOUNT VALUES ("5890-1970-7706-8912", NULL);
""")

# 5. Create BANK_ACCOUNT_TRANSACTION table
cursor.execute("""
CREATE TABLE BANK_ACCOUNT_TRANSACTION (
    Account_Number VARCHAR(19),
    Transaction_amount DECIMAL(18,2),
    Transcation_channel VARCHAR(18),
    Province VARCHAR(3),
    Transaction_Date DATE
);
""")

# 6. Insert transaction data
cursor.executescript("""
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-3456", -2000, "ATM withdrawl", "CA", "2020-01-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -4000, "POS-Walmart", "MN", "2020-02-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -1600, "UPI transfer", "MN", "2020-01-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -6000, "Bankers cheque", "CA", "2020-03-23");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -3000, "Net banking", "CA", "2020-04-24");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", 23000, "cheque deposit", "MN", "2020-03-15");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5000-1700-6091", 40000, "ECS transfer", "NY", "2020-02-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5000-1700-7791", 40000, "ECS transfer", "NY", "2020-02-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-3401", 8000, "Cash Deposit", "NY", "2020-01-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5102", -6500, "ATM withdrawal", "NY", "2020-03-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5698", -9000, "Cash Deposit", "NY", "2020-03-27");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-9977", 50000, "ECS transfer", "NY", "2020-01-16");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -5000, "POS-Walmart", "NY", "2020-02-17");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -8000, "Shopping Cart", "MN", "2020-03-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -2500, "Shopping Cart", "MN", "2020-04-21");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5800-1700-9800-7755", -9000, "POS-Walmart", "MN", "2020-04-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5890-1970-7706-8912", -11000, "Shopping Cart", "NY", "2020-03-12");
""")

conn.commit()

# 7. Query: transactions in March & April 2020
query = """
SELECT
    a.AccountNumber,
    a.Account_Balance AS Balance_Amount,
    t.Transaction_amount,
    t.Transaction_Date
FROM ACCOUNT a
JOIN BANK_ACCOUNT_TRANSACTION t
    ON a.AccountNumber = t.Account_Number
WHERE strftime('%Y-%m', t.Transaction_Date) IN ('2020-03', '2020-04');
"""

cursor.execute(query)
rows = cursor.fetchall()

# 8. Print results safely handling NULLs
print(f"{'AccountNumber':<20} {'Balance_Amount':<15} {'Transaction_amount':<20} {'Transaction_Date'}")
print("-" * 80)
for row in rows:
    acc = row[0]
    bal = row[1] if row[1] is not None else 'NULL'
    amt = row[2] if row[2] is not None else 'NULL'
    date = row[3] if row[3] is not None else 'NULL'
    print(f"{acc:<20} {bal:<15} {amt:<20} {date}")

conn.close()


4) Print all of the customer id, account number,  balance_amount, transaction_amount , Transaction_Date  from bank_customer, Bank_Account_Details and bank_account_transaction tables where excluding all of their transactions in march, 2020  month (22 Rows)

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Bank_Account_Details;")

cursor.execute("""
CREATE TABLE Bank_Account_Details (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(18,2)
);
""")

# Insert sample account data
cursor.executescript("""
INSERT INTO Bank_Account_Details VALUES ("4000-1956-3456", 1, 0);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-2001", 1, NULL);
INSERT INTO Bank_Account_Details VALUES ("5000-1700-6091", 2, 15000);
INSERT INTO Bank_Account_Details VALUES ("5000-1700-7791", 2, NULL);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5102", 3, -6500);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5698", 3, -9000);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-9977", 4, 50000);
INSERT INTO Bank_Account_Details VALUES ("9000-1700-7777-4321", 5, NULL);
INSERT INTO Bank_Account_Details VALUES ("5800-1700-9800-7755", 5, NULL);
INSERT INTO Bank_Account_Details VALUES ("5890-1970-7706-8912", 5, NULL);
""")


In [ ]:
cursor.execute("DROP TABLE IF EXISTS BANK_ACCOUNT_TRANSACTION;")

cursor.execute("""
CREATE TABLE BANK_ACCOUNT_TRANSACTION (
    Account_Number VARCHAR(19),
    Transaction_amount DECIMAL(18,2),
    Transcation_channel VARCHAR(18),
    Province VARCHAR(3),
    Transaction_Date DATE
);
""")

# Insert transaction data
cursor.executescript("""
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-3456", -2000, "ATM withdrawl", "CA", "2020-01-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -4000, "POS-Walmart", "MN", "2020-02-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -1600, "UPI transfer", "MN", "2020-01-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -6000, "Bankers cheque", "CA", "2020-03-23");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -3000, "Net banking", "CA", "2020-04-24");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", 23000, "cheque deposit", "MN", "2020-03-15");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5102", -6500, "ATM withdrawal", "NY", "2020-03-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5698", -9000, "Cash Deposit", "NY", "2020-03-27");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -8000, "Shopping Cart", "MN", "2020-03-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -2500, "Shopping Cart", "MN", "2020-04-21");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5800-1700-9800-7755", -9000, "POS-Walmart", "MN", "2020-04-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5890-1970-7706-8912", -11000, "Shopping Cart", "NY", "2020-03-12");
""")


In [ ]:
cursor.execute("DROP TABLE IF EXISTS BANK_ACCOUNT_TRANSACTION;")

cursor.execute("""
CREATE TABLE BANK_ACCOUNT_TRANSACTION (
    Account_Number VARCHAR(19),
    Transaction_amount DECIMAL(18,2),
    Transcation_channel VARCHAR(18),
    Province VARCHAR(3),
    Transaction_Date DATE
);
""")

# Insert transaction data
cursor.executescript("""
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-3456", -2000, "ATM withdrawl", "CA", "2020-01-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -4000, "POS-Walmart", "MN", "2020-02-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -1600, "UPI transfer", "MN", "2020-01-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -6000, "Bankers cheque", "CA", "2020-03-23");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -3000, "Net banking", "CA", "2020-04-24");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", 23000, "cheque deposit", "MN", "2020-03-15");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5102", -6500, "ATM withdrawal", "NY", "2020-03-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5698", -9000, "Cash Deposit", "NY", "2020-03-27");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -8000, "Shopping Cart", "MN", "2020-03-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -2500, "Shopping Cart", "MN", "2020-04-21");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5800-1700-9800-7755", -9000, "POS-Walmart", "MN", "2020-04-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5890-1970-7706-8912", -11000, "Shopping Cart", "NY", "2020-03-12");
""")


In [ ]:
import sqlite3

# 1. Connect to database
conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# 2. Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS bank_customer;")
cursor.execute("DROP TABLE IF EXISTS Bank_Account_Details;")
cursor.execute("DROP TABLE IF EXISTS BANK_ACCOUNT_TRANSACTION;")

# 3. Create bank_customer table
cursor.execute("""
CREATE TABLE bank_customer (
    CustomerID INTEGER PRIMARY KEY,
    CustomerName VARCHAR(50)
);
""")

cursor.executescript("""
INSERT INTO bank_customer VALUES (1, "Ramesh");
INSERT INTO bank_customer VALUES (2, "Suresh");
INSERT INTO bank_customer VALUES (3, "Geeta");
INSERT INTO bank_customer VALUES (4, "Anita");
INSERT INTO bank_customer VALUES (5, "Rajesh");
""")

# 4. Create Bank_Account_Details table
cursor.execute("""
CREATE TABLE Bank_Account_Details (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(18,2)
);
""")

cursor.executescript("""
INSERT INTO Bank_Account_Details VALUES ("4000-1956-3456", 1, 0);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-2001", 1, NULL);
INSERT INTO Bank_Account_Details VALUES ("5000-1700-6091", 2, 15000);
INSERT INTO Bank_Account_Details VALUES ("5000-1700-7791", 2, NULL);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5102", 3, -6500);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5698", 3, -9000);
INSERT INTO Bank_Account_Details VALUES ("4000-1956-9977", 4, 50000);
INSERT INTO Bank_Account_Details VALUES ("9000-1700-7777-4321", 5, NULL);
INSERT INTO Bank_Account_Details VALUES ("5800-1700-9800-7755", 5, NULL);
INSERT INTO Bank_Account_Details VALUES ("5890-1970-7706-8912", 5, NULL);
""")

# 5. Create BANK_ACCOUNT_TRANSACTION table
cursor.execute("""
CREATE TABLE BANK_ACCOUNT_TRANSACTION (
    Account_Number VARCHAR(19),
    Transaction_amount DECIMAL(18,2),
    Transcation_channel VARCHAR(18),
    Province VARCHAR(3),
    Transaction_Date DATE
);
""")

cursor.executescript("""
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-3456", -2000, "ATM withdrawl", "CA", "2020-01-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -4000, "POS-Walmart", "MN", "2020-02-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -1600, "UPI transfer", "MN", "2020-01-19");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -6000, "Bankers cheque", "CA", "2020-03-23");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", -3000, "Net banking", "CA", "2020-04-24");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-2001", 23000, "cheque deposit", "MN", "2020-03-15");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5102", -6500, "ATM withdrawal", "NY", "2020-03-14");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("4000-1956-5698", -9000, "Cash Deposit", "NY", "2020-03-27");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -5000, "POS-Walmart", "NY", "2020-02-17");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -8000, "Shopping Cart", "MN", "2020-03-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("9000-1700-7777-4321", -2500, "Shopping Cart", "MN", "2020-04-21");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5800-1700-9800-7755", -9000, "POS-Walmart", "MN", "2020-04-13");
INSERT INTO BANK_ACCOUNT_TRANSACTION VALUES ("5890-1970-7706-8912", -11000, "Shopping Cart", "NY", "2020-03-12");
""")

conn.commit()

# 6. Query: Exclude all March 2020 transactions
query = """
SELECT
    c.CustomerID,
    a.AccountNumber,
    a.Account_Balance AS Balance_Amount,
    t.Transaction_amount,
    t.Transaction_Date
FROM bank_customer c
JOIN Bank_Account_Details a
    ON c.CustomerID = a.CustomerID
JOIN BANK_ACCOUNT_TRANSACTION t
    ON a.AccountNumber = t.Account_Number
WHERE strftime('%Y-%m', t.Transaction_Date) != '2020-03';
"""

cursor.execute(query)
rows = cursor.fetchall()

# 7. Print results safely
print(f"{'CustomerID':<12} {'AccountNumber':<20} {'Balance_Amount':<15} {'Transaction_amount':<20} {'Transaction_Date'}")
print("-" * 95)
for row in rows:
    cust = row[0]
    acc = row[1]
    bal = row[2] if row[2] is not None else 'NULL'
    amt = row[3] if row[3] is not None else 'NULL'
    date = row[4] if row[4] is not None else 'NULL'
    print(f"{cust:<12} {acc:<20} {bal:<15} {amt:<20} {date}")

conn.close()


5) Print only the customer id, account_number, balance_amount,transaction_amount ,transaction_date who did transactions during the first quarter. Do not display the accounts if they have not done any transactions in the first quarter.(16 Rows)

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# SQL query inside a string
query = """
SELECT
    c.CustomerID,
    a.AccountNumber,
    a.Account_Balance AS Balance_Amount,
    t.Transaction_amount,
    t.Transaction_Date
FROM bank_customer c
JOIN Bank_Account_Details a
    ON c.CustomerID = a.CustomerID
JOIN BANK_ACCOUNT_TRANSACTION t
    ON a.AccountNumber = t.Account_Number
WHERE strftime('%m', t.Transaction_Date) IN ('01', '02', '03');
"""

# Execute query
cursor.execute(query)
rows = cursor.fetchall()

# Print results safely
print(f"{'CustomerID':<12} {'AccountNumber':<20} {'Balance_Amount':<15} {'Transaction_amount':<20} {'Transaction_Date'}")
print("-" * 95)

for row in rows:
    cust = row[0]
    acc = row[1]
    bal = row[2] if row[2] is not None else 'NULL'
    amt = row[3] if row[3] is not None else 'NULL'
    date = row[4] if row[4] is not None else 'NULL'
    print(f"{cust:<12} {acc:<20} {bal:<15} {amt:<20} {date}")

conn.close()


6) Print account_number, Event and Customer_message from BANK_CUSTOMER_MESSAGES and Bank_Account_Details to display an “Adhoc" Event for all customers who have  “SAVINGS" account_type account.(8 Rows)

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Drop old table (optional)
cursor.execute("DROP TABLE IF EXISTS Bank_Account_Details;")

# Recreate table with Account_type
cursor.execute("""
CREATE TABLE Bank_Account_Details (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(18,2),
    Account_type VARCHAR(10)
);
""")

# Insert sample accounts with SAVINGS / CURRENT types
cursor.executescript("""
INSERT INTO Bank_Account_Details VALUES ("4000-1956-3456", 1, 0, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-2001", 1, NULL, "CURRENT");
INSERT INTO Bank_Account_Details VALUES ("5000-1700-6091", 2, 15000, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("5000-1700-7791", 2, NULL, "CURRENT");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5102", 3, -6500, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5698", 3, -9000, "CURRENT");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-9977", 4, 50000, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("9000-1700-7777-4321", 5, NULL, "CURRENT");
""")

conn.commit()
conn.close()


In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# ----------------------------
# 1️⃣ Create Bank_Account_Details
# ----------------------------
cursor.execute("DROP TABLE IF EXISTS Bank_Account_Details;")
cursor.execute("""
CREATE TABLE Bank_Account_Details (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(18,2),
    Account_type VARCHAR(10)
);
""")

# Insert sample accounts
cursor.executescript("""
INSERT INTO Bank_Account_Details VALUES ("4000-1956-3456", 1, 0, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-2001", 1, NULL, "CURRENT");
INSERT INTO Bank_Account_Details VALUES ("5000-1700-6091", 2, 15000, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("5000-1700-7791", 2, NULL, "CURRENT");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5102", 3, -6500, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5698", 3, -9000, "CURRENT");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-9977", 4, 50000, "SAVINGS");
INSERT INTO Bank_Account_Details VALUES ("9000-1700-7777-4321", 5, NULL, "CURRENT");
""")

# ----------------------------
# 2️⃣ Create BANK_CUSTOMER_MESSAGES
# ----------------------------
cursor.execute("DROP TABLE IF EXISTS BANK_CUSTOMER_MESSAGES;")
cursor.execute("""
CREATE TABLE BANK_CUSTOMER_MESSAGES (
    Event VARCHAR(24),
    Customer_message VARCHAR(75),
    Notice_delivery_mode VARCHAR(15)
);
""")

# Insert Adhoc message
cursor.executescript("""
INSERT INTO BANK_CUSTOMER_MESSAGES VALUES
('Adhoc', 'All Banks are closed due to announcement of National strike', 'mobile');
INSERT INTO BANK_CUSTOMER_MESSAGES VALUES
('Transaction Limit', 'Only limited withdrawals per card are allowed from ATM machines', 'mobile');
""")

conn.commit()

# ----------------------------
# 3️⃣ Query: Adhoc Event for SAVINGS Accounts
# ----------------------------
query = """
SELECT
    a.AccountNumber,
    m.Event,
    m.Customer_message
FROM Bank_Account_Details a
JOIN BANK_CUSTOMER_MESSAGES m
    ON m.Event = 'Adhoc'
WHERE a.Account_type = 'SAVINGS';
"""

cursor.execute(query)
rows = cursor.fetchall()

# ----------------------------
# 4️⃣ Print results
# ----------------------------
print(f"{'AccountNumber':<20} {'Event':<10} {'Customer_message'}")
print("-" * 70)
for row in rows:
    print(f"{row[0]:<20} {row[1]:<10} {row[2]}")

conn.close()


7) Print all Customer_id, Account_Number, Account_type, and display deducted balance_amount by  subtracting only negative transaction_amounts for Relationship_type ="P" ( P - means  Primary , S - means Secondary ) .(27 Rows)

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS Bank_Account_Details;")
cursor.execute("DROP TABLE IF EXISTS bank_account_transaction;")

# Create Bank_Account_Details with Relationship_type
cursor.execute("""
CREATE TABLE Bank_Account_Details (
    AccountNumber VARCHAR(19) PRIMARY KEY,
    CustomerID INTEGER,
    Account_Balance DECIMAL(18,2),
    Account_type VARCHAR(10),
    Relationship_type VARCHAR(1)
);
""")

# Insert sample accounts
cursor.executescript("""
INSERT INTO Bank_Account_Details VALUES ("4000-1956-3456", 1, 10000, "SAVINGS", "P");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-2001", 1, 20000, "CURRENT", "S");
INSERT INTO Bank_Account_Details VALUES ("5000-1700-6091", 2, 5000, "SAVINGS", "P");
INSERT INTO Bank_Account_Details VALUES ("5000-1700-7791", 2, 15000, "CURRENT", "S");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5102", 3, 12000, "SAVINGS", "P");
INSERT INTO Bank_Account_Details VALUES ("4000-1956-5698", 3, 8000, "CURRENT", "S");
""")

# Create bank_account_transaction table
cursor.execute("""
CREATE TABLE bank_account_transaction (
    Account_Number VARCHAR(19),
    Transaction_amount DECIMAL(18,2),
    Transcation_channel VARCHAR(18),
    Province VARCHAR(3),
    Transaction_Date DATE
);
""")

# Insert transactions (some negative, some positive)
cursor.executescript("""
INSERT INTO bank_account_transaction VALUES ("4000-1956-3456", -2000, "ATM withdrawl", "CA", "2020-01-13");
INSERT INTO bank_account_transaction VALUES ("4000-1956-3456", 3000, "Deposit", "CA", "2020-01-15");
INSERT INTO bank_account_transaction VALUES ("5000-1700-6091", -1000, "POS-Walmart", "MN", "2020-02-14");
INSERT INTO bank_account_transaction VALUES ("5000-1700-6091", 2000, "Deposit", "MN", "2020-02-15");
INSERT INTO bank_account_transaction VALUES ("4000-1956-5102", -5000, "ATM withdrawl", "NY", "2020-03-10");
INSERT INTO bank_account_transaction VALUES ("4000-1956-5102", 1000, "Deposit", "NY", "2020-03-12");
""")

conn.commit()


In [ ]:
query = """
SELECT
    b.CustomerID,
    b.AccountNumber,
    b.Account_type,
    b.Account_Balance - IFNULL(SUM(CASE WHEN t.Transaction_amount < 0 THEN t.Transaction_amount ELSE 0 END), 0) AS Deducted_Balance
FROM Bank_Account_Details b
LEFT JOIN bank_account_transaction t
    ON b.AccountNumber = t.Account_Number
WHERE b.Relationship_type = 'P'
GROUP BY b.CustomerID, b.AccountNumber, b.Account_type, b.Account_Balance;
"""

cursor.execute(query)
rows = cursor.fetchall()

# Print results
print(f"{'CustomerID':<12} {'AccountNumber':<20} {'Account_type':<12} {'Deducted_Balance'}")
print("-" * 60)
for row in rows:
    print(f"{row[0]:<12} {row[1]:<20} {row[2]:<12} {row[3]}")

conn.close()


8:
a) Display records of All Accounts , their Account_types, the transaction amount.


In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# SQL query as a string
query = """
SELECT
    b.AccountNumber,
    b.Account_type,
    t.Transaction_amount
FROM Bank_Account_Details b
LEFT JOIN bank_account_transaction t
    ON b.AccountNumber = t.Account_Number
ORDER BY b.AccountNumber;
"""

# Execute query
cursor.execute(query)
rows = cursor.fetchall()

# Print output
print(f"{'AccountNumber':<20} {'Account_type':<12} {'Transaction_amount'}")
print("-" * 50)
for row in rows:
    print(f"{row[0]:<20} {row[1]:<12} {row[2]}")

conn.close()


8: b) Along with first step, Display other columns with corresponding linking account number, account types (15 Rows)

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Query to join tables and display all columns for each account
query = """
SELECT
    b.AccountNumber,
    b.Account_type,
    b.CustomerID,
    t.Transaction_amount,
    t.Transaction_Date,
    t.Province,
    t.Transcation_channel
FROM Bank_Account_Details b
LEFT JOIN bank_account_transaction t
    ON b.AccountNumber = t.Account_Number
ORDER BY b.AccountNumber;
"""

cursor.execute(query)
rows = cursor.fetchall()

# Print output with headers
print(f"{'AccountNumber':<20} {'Account_type':<12} {'CustomerID':<10} {'Transaction_amount':<18} {'Transaction_Date':<12} {'Province':<8} {'Channel'}")
print("-" * 95)
for row in rows:
    print(f"{row[0]:<20} {row[1]:<12} {row[2]:<10} {str(row[3]):<18} {str(row[4]):<12} {str(row[5]):<8} {row[6]}")

conn.close()


9:
a) Display records of All Accounts , their Account_types, the transaction amount.

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# SQL query
query = """
SELECT
    b.AccountNumber,
    b.Account_type,
    t.Transaction_amount
FROM Bank_Account_Details b
LEFT JOIN bank_account_transaction t
    ON b.AccountNumber = t.Account_Number
ORDER BY b.AccountNumber;
"""

cursor.execute(query)
rows = cursor.fetchall()

# Print output
print(f"{'AccountNumber':<20} {'Account_type':<12} {'Transaction_amount'}")
print("-" * 50)
for row in rows:
    print(f"{row[0]:<20} {row[1]:<12} {row[2]}")

conn.close()


9: b) Along with first step, Display other columns with corresponding linking account number, account types

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Query to join tables and display all columns
query = """
SELECT
    b.AccountNumber,
    b.Account_type,
    b.CustomerID,
    t.Transaction_amount,
    t.Transaction_Date,
    t.Province,
    t.Transcation_channel
FROM Bank_Account_Details b
LEFT JOIN bank_account_transaction t
    ON b.AccountNumber = t.Account_Number
ORDER BY b.AccountNumber;
"""

cursor.execute(query)
rows = cursor.fetchall()

# Print output neatly
print(f"{'AccountNumber':<20} {'Account_type':<12} {'CustomerID':<10} {'Transaction_amount':<18} {'Transaction_Date':<12} {'Province':<8} {'Channel'}")
print("-" * 95)
for row in rows:
    print(f"{row[0]:<20} {row[1]:<12} {row[2]:<10} {str(row[3]):<18} {str(row[4]):<12} {str(row[5]):<8} {row[6]}")

conn.close()


9: c) After retrieving all records of accounts and their linked accounts, display the   transaction amount of accounts appeared  in another column.(26 Rows)

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Query: join account details with transactions
query = """
SELECT
    b.AccountNumber,
    b.Account_type,
    b.CustomerID,
    t.Transaction_amount AS TransactionAmount,
    t.Transaction_Date,
    t.Province,
    t.Transcation_channel
FROM Bank_Account_Details b
LEFT JOIN bank_account_transaction t
    ON b.AccountNumber = t.Account_Number
ORDER BY b.AccountNumber;
"""

cursor.execute(query)
rows = cursor.fetchall()

# Print output neatly
print(f"{'AccountNumber':<20} {'Account_type':<12} {'CustomerID':<10} {'TransactionAmount':<18} {'Transaction_Date':<12} {'Province':<8} {'Channel'}")
print("-" * 95)
for row in rows:
    print(f"{row[0]:<20} {row[1]:<12} {row[2]:<10} {str(row[3]):<18} {str(row[4]):<12} {str(row[5]):<8} {row[6]}")

conn.close()


10) Display all saving account holders have “Add-on Credit Cards" and “Credit cards" (3 Rows)

In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Create table for Customer Cards
cursor.execute("""
CREATE TABLE Bank_Customer_Cards (
    AccountNumber VARCHAR(19),
    Card_type VARCHAR(30)
)
""")

# Insert sample data
cards_data = [
    ('4000-1956-3456', 'Credit Card'),
    ('4000-1956-3456', 'Add-on Credit Card'),
    ('4000-1956-5102', 'Credit Card')
]

cursor.executemany("INSERT INTO Bank_Customer_Cards VALUES (?, ?)", cards_data)
conn.commit()
conn.close()


In [ ]:
import sqlite3

conn = sqlite3.connect("bank.db")
cursor = conn.cursor()

# Create table for customer cards
cursor.execute("""
CREATE TABLE IF NOT EXISTS Bank_Customer_Cards (
    AccountNumber VARCHAR(19),
    Card_type VARCHAR(30)
)
""")

# Insert sample data
cards_data = [
    ('4000-1956-3456', 'Credit Card'),
    ('4000-1956-3456', 'Add-on Credit Card'),
    ('4000-1956-5102', 'Credit Card')
]

cursor.executemany("INSERT INTO Bank_Customer_Cards VALUES (?, ?)", cards_data)
conn.commit()


In [ ]:
query = """
SELECT DISTINCT b.CustomerID, b.AccountNumber, c.Card_type
FROM Bank_Account_Details b
JOIN Bank_Customer_Cards c
    ON b.AccountNumber = c.AccountNumber
WHERE b.Account_type = 'SAVINGS'
  AND c.Card_type IN ('Add-on Credit Card', 'Credit Card')
ORDER BY b.CustomerID;
"""

cursor.execute(query)
rows = cursor.fetchall()

# Print output
print(f"{'CustomerID':<12} {'AccountNumber':<20} {'Card_type'}")
print("-" * 50)
for row in rows:
    print(f"{row[0]:<12} {row[1]:<20} {row[2]}")

conn.close()


11)  Display  records of “SAVINGS” account linked with “Credit cards" account_type and its credit
aggregate sum of transaction amount. (1 Row)


In [ ]:
# SQL query wrapped in triple quotes
query = """
SELECT
    'SAVINGS' AS account_type,
    'Credit Cards' AS card_account_type,
    a.Account_Balance AS credit,
    SUM(t.Amount) AS total_transaction_amount
FROM Account a
JOIN Bank_Customer_Cards cc
    ON a.AccountNumber = cc.AccountNumber
JOIN Transactions t
    ON a.AccountNumber = t.AccountNumber
WHERE cc.Card_type = 'Credit Cards'
GROUP BY a.AccountNumber;
"""

cursor.execute(query)
row = cursor.fetchone()  # fetch one row

print("Account Type:", row[0])
print("Card Account Type:", row[1])
print("Credit:", row[2])
print("Total Transaction Amount:", row[3])
